无约束优化问题：常见的算法包括,'BFGS','Newton-CG','L-BFGS-B'

scipy.optimize.minimize

Proximal Gradient
模型假设:
$$\min F(x)=f(x)+g(x)$$
Assumption:

1)g is proper closed and convex

2)f is proper and closed, dom(f) is convex, dom(f)$\subseteq$ int(dom(f)), and f is $L_f$-smooth over int(dom(f)). 

3)问题的解是非空的


Proximal Gradient Method:

Initialization: pick $x^0 \in int(dom(f))$

General Step: for any $k=0,1,2,...$ excute the following steps:

* a) pick $L_k>0$

* b) set $x^{k+1}=prox_{\frac{1}{L_k}g}(x^k-\frac{1}{L_k}\nabla f(x^k))$.

这里给出prox算子的定义
    $prox_f(x)=\arg \min_{u} \{ f(u)+\frac{1}{2}||u-x||^2\}.$

用Proximal Gradient方法解决Lasso问题：
$$\min \frac{1}{2} ||Y-X^T\beta||^2+C ||\beta||_1$$

In [10]:
def F(y,X,beta,C):
    return 1/2*np.power(y-np.dot(X,beta),2).sum()+C*(np.abs(beta).sum())
def f(y,X,beta):
    return 1/2*np.power(y-np.dot(X,beta),2).sum()
def subf(y,X,beta):
    return np.dot(X.T,np.dot(X,beta)-y)
def prox_L1(beta,C):
    return np.sign(beta)*np.maximum(np.abs(beta)-C,0)

In [11]:
#Simulate the data
import numpy as np
import pandas as pd
n=500
p=200
s=20
beta=np.append(np.random.normal(0,1,s),np.zeros(p-s))
X=np.random.normal(0,1,(n,p))
y=np.dot(X,beta)+0.1*np.random.normal(0,1,n)

In [12]:
def PGD_LASSO(y,X,beta,C=10,l=1,eta=1.2,maxiter=10000,conv=0.0001):
    opt_F=[]
    for i in range(maxiter):
        grad_x=subf(y,X,beta)
        while True:
            z=prox_L1(beta-grad_x/l,C/l)
            if  f(y,X,z)<f(y,X,beta)+np.dot(grad_x,z-beta)+l/2*np.power(z-beta,2).sum():
                break
            l=eta*l
        beta=z
        opt_F.append(F(y,X,beta,C))
        if i>1 and np.abs(opt_F[i]-opt_F[i-1])<conv:
            print('convergence reached after {} iterations'.format(i))
            break
        if i==(maxiter-1):
            print('algorithm fails to converge')
    return beta,opt_F
beta,opt_F=PGD_LASSO(y,X,beta=np.random.normal(0,1,p))

convergence reached after 26 iterations


带约束的优化问题

1)QP问题可以直接用拉格朗日法求解

2）ADMM算法
$$H_{opt}=\min \{ H(x,z)=h_1(x)+h_2(z): Ax+Bz=c\}$$.

这里我们假设: $h_1$和$h_2$ are proper closed and convex functions.


ADMM

Initialization : $x^0\in R^n, z^0 \in R^p, y^0\in R^m ,\rho>0$

General Step: for any $k=0,1,...$ excute the following:

(a) $x^{k+1}\in \arg\min_x \{ h_1(x)+\frac{\rho}{2} ||Ax+Bz^k-c+\frac{1}{\rho}y^k||^2\}$

(b)$z^{k+1}\in \arg\min_z \{ h_2(z)+\frac{\rho}{2} ||Ax^{k+1}+Bz-c+\frac{1}{\rho}y^k||^2\}$

(c) $y^{k+1}=y^k+\rho(Ax^{k+1}+Bz^{k+1}-c)$

AD-LPMM

Initialization: $x^0\in R^n, z^0 \in R^p, y^0\in R^m ,\rho>0, \alpha \ge \rho \lambda_{max}(A^TA),\beta\ge \rho\lambda_{max}(B^TB)$

General Step: for any $k=0,1,...$ excute the following:

(a) $x^{k+1}=prox_{\frac{1}{\alpha}h_1} [x^k-\frac{\rho}{\alpha}A^T(Ax^k+Bz^k-c+\frac{1}{\rho}y^k)]$

(b) $z^{k+1}=prox_{\frac{1}{\beta}h_2}[z^k-\frac{\rho}{\alpha}B^T(Ax^{k+1}+Bz^k-c+\frac{1}{\rho}y^k)]$

(c) $y^{k+1}=y^k+\rho(Ax^{k+1}+Bz^{k+1}-c)$